In [ ]:
import numpy as np
import pandas as pd

import matplotlib, cv2
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pdf
import matplotlib.patches as pch
import eleanor_constants as EL

matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
matplotlib.rcParams['svg.fonttype'] = 'path'
%matplotlib inline

savename = "./figures/S2.pdf"

In [ ]:
example_file = "./data/fluorescein/170804-experiment-01/crop_experiments/170804-01-02-00-cropped-experiment.jpg"

test_image = cv2.imread(example_file)
b = test_image[:,:,0]
g = test_image[:,:,1]
r = test_image[:,:,2]

hsv = cv2.cvtColor(test_image, cv2.COLOR_BGR2HSV)
h = hsv[:,:,0]
s = hsv[:,:,1]
v = hsv[:,:,2]

labels = ["R", "G", "B", "H", "S", "V"]
colors = [r, g, b, h, s, v]
var_df = pd.read_csv("./data/fluorescein/color_spaces_variance_results.csv")

In [ ]:
df_distance = pd.read_csv("./data/fluorescein/distance_concentration_map_fitted.csv")
x_dist = df_distance["distance_mm"]
y_dist = df_distance["concentration"]

df_standard = pd.read_csv("./data/fluorescein/standardization_saturation_stats.csv")
x_standard = df_standard["concentration"].values
y_standard = df_standard["mean"].values
err_standard = df_standard["std_dev"].values

cutoff_df = pd.read_csv("./data/fluorescein/0min_concentration_values_cutoff_calculation.csv")
cutoff_values = cutoff_df["values_0min"].values.flatten()

df_calc = pd.read_csv("./data/fluorescein/cutoff_concentration.csv")
cutoff_xmax = df_calc['max_data'].values[0]

In [ ]:
times = ['ss-ss', '00-00', '01-00', '02-00', '03-00', '04-00', '05-00', 
         '06-00', '07-00', '08-00', '09-00', '10-00', '11-00',
         '12-00', '13-00', '14-00', '15-00']

# Experimental treatments with larvae
df = pd.read_csv("./data/fluorescein/files_larvae.csv")
exps = df["larvae_experiments"].values
filenames, data = [], []

for time in times:  
    for exp in exps:
        date = exp.split('-')[0]
        num = exp.split('-')[-1]
        f_temp = './data/fluorescein/'+exp+'/1mm_csv_concentrations/'+date+'-'+num+'-'+time+'_concentrations.csv'
        filenames.append(f_temp)
        df = pd.read_csv(f_temp)
        data.append(df)

In [ ]:
cmap='bone_r'
cmap2='RdPu'
rows, columns = 17+1+6+4, len(exps)+2

fig = plt.figure(figsize=(14.5, 14))
grid = matplotlib.gridspec.GridSpec(rows, columns)
        
axA = fig.add_subplot(grid[0:2, 0:2], facecolor="None", aspect="equal")
axB = fig.add_subplot(grid[0:2, 2:4], facecolor="None", aspect="equal")
axC = fig.add_subplot(grid[0:2, 4:6], facecolor="None", aspect="equal")
axD = fig.add_subplot(grid[0:2, 6:8], facecolor="None", aspect="equal")
axE = fig.add_subplot(grid[0:2, 8:10], facecolor="None", aspect="equal")
axF = fig.add_subplot(grid[0:2, 10:], facecolor="None", aspect="equal")
axG = fig.add_subplot(grid[2:4, :], facecolor="None")

ax1 = fig.add_subplot(grid[4:21, 0], facecolor="None") # Time axis for mean concentrations
ax12 = fig.add_subplot(grid[4:21, 2], facecolor="None") # Time axis for individual concentrations

axX1 = fig.add_subplot(grid[22:, 0:2], facecolor="None")
axX2 = fig.add_subplot(grid[22:, 2:3], facecolor="None")
axY = fig.add_subplot(grid[22:, 4:7], facecolor="None")
axZ = fig.add_subplot(grid[22:, 8:], facecolor="None")
labelax = fig.add_subplot(grid[22:, 0:3], facecolor="None")

for i, (ax, label) in enumerate(zip([axA, axB, axC, axD, axE, axF], labels)): 
    ax.spines['bottom'].set_color(EL.c_greyax)
    ax.spines['top'].set_color(EL.c_greyax) 
    ax.spines['right'].set_color(EL.c_greyax)
    ax.spines['left'].set_color(EL.c_greyax)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(colors[i], cmap=cmap2, vmin=0, vmax=255)
    ax.set_title(label, loc="left")

axG.axis("off")

a=np.outer(np.ones(10),np.arange(0,1,0.0001))
img = axG.imshow(a, aspect='auto', cmap=cmap, origin="lower")
img.set_visible(False)
cax = fig.add_axes([2.5/12, 0.89, (0.45*9.5/12), 0.0075]) 
cbar = plt.colorbar(img, cax=cax, orientation="horizontal", ticks=np.arange(0, 1.2, 0.2))
cbar.ax.set_xlabel("Dye Concentration (\%)")
cbar.ax.set_xticklabels(np.arange(0, 101, 20))

img = axG.imshow(a, aspect='auto', cmap=cmap2, origin="lower")
img.set_visible(False)
cax = fig.add_axes([2.5/12+(0.5*9.5/12), 0.89, (0.45*9.5/12), 0.0075]) 
cbar = plt.colorbar(img, cax=cax, orientation="horizontal", ticks=np.arange(0, 1.2, 0.2))
cbar.ax.set_xlabel("Channel encoding (0-255)")
cbar.ax.set_xticklabels(np.arange(0, 256, 50))

for ax in [axX1, axX2, axY, axZ]:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
bins = np.arange(0, 100, 0.1)
axY.hist(cutoff_values, bins=bins, normed=True, color=matplotlib.cm.get_cmap(cmap)(0.5), 
         alpha=0.75, lw=1, edgecolor="#ffffff")
axY.axvline(x=df_calc['cutoff_concentration'].values[0], ymin=0, ymax=1, label="Data cutoff", lw=1, 
            c=matplotlib.cm.get_cmap(cmap)(0.25))
axY.axvline(x=cutoff_xmax, ymin=0, ymax=1, label="Data maximum", lw=1, 
            c=matplotlib.cm.get_cmap(cmap)(0.75))
axY.set_xscale('log')
axY.set_ylim(0, 8)
axY.set_xlim(right=100)
axY.legend(frameon=False, loc='upper right', handletextpad=0.5, 
          bbox_to_anchor=(-0.1, 0, 1, 1.125), ncol=2, columnspacing=0.75)
axY.set_ylabel("Frequency (normalized)")
axY.set_xlabel("Concentration (\%)")

for axX in [axX1, axX2]:
    axX.spines['right'].set_visible(False)
    axX.spines['top'].set_visible(False)
    axX.set_ylim(0, 255)
    axX.set_yticks(np.arange(0, 256, 50))
    axX.set_yticks(np.arange(0, 256, 25), minor=True)
    axX.errorbar(x_standard, y_standard, yerr=err_standard, alpha=0.5, color=matplotlib.cm.get_cmap(cmap)(0.5))
    axX.fill_between(x_standard, y_standard-err_standard, y_standard+err_standard, alpha=0.25, 
                    color=matplotlib.cm.get_cmap(cmap)(0.5))
    
axX1.plot(x_standard[0:11], y_standard[0:11], color=matplotlib.cm.get_cmap(cmap)(0.5), clip_on=False)
axX2.plot(x_standard[11:], y_standard[11:], color=matplotlib.cm.get_cmap(cmap)(0.5), clip_on=False)
axX1.scatter(x_standard[0:11], y_standard[0:11], s=EL.s, color=matplotlib.cm.get_cmap(cmap)(0.5), clip_on=False)
axX2.scatter(x_standard[11:], y_standard[11:], s=EL.s, color=matplotlib.cm.get_cmap(cmap)(0.5), clip_on=False)
axX1.set_xlim(0, 21)
axX2.set_xlim(20, 100)
axX2.spines['left'].set_visible(False)
axX2.set_yticks([])
axX2.set_yticks([], minor=True)
axX2.set_xticks(np.arange(20, 100, 10), minor=True)
axX1.set_xticks(np.arange(0, 21, 10))

labelax.set_xlabel("Concentration (\%)", labelpad=20)
labelax.set_ylabel("Color encoding (S)", labelpad=30)
labelax.spines['bottom'].set_visible(False)
labelax.spines['left'].set_visible(False)
labelax.spines['top'].set_visible(False)
labelax.spines['right'].set_visible(False)
labelax.set_xticks([])
labelax.set_yticks([])

axZ.scatter(x_dist, y_dist, s=4, lw=0, color=matplotlib.cm.get_cmap(cmap)(0.5), 
            marker="o", alpha=0.5, clip_on=False)
a = -0.08046986117494856
b = 4.897119830335053
x_sim = np.arange(0, np.hypot(80, 30), 1)
y_sim = [np.exp(b)*np.exp(a*g) for g in x_sim]
axZ.plot(x_sim, y_sim, alpha=0.75, lw=1, color=matplotlib.cm.get_cmap(cmap)(0.999999))
axZ.set_yscale("log")
axZ.set_ylim(10**-2, 100)
axZ.set_xlim(0, np.hypot(80, 30))
axZ.set_xticks(np.arange(0, 81, 20))
axZ.set_xticks(np.arange(0, 81, 10), minor=True)
axZ.set_xlabel("Distance from stimulus source (mm)")
axZ.set_ylabel("Concentration (\%)")
axZ.text(7.5, 0.05, "${y = e^{4.9} \cdot e^{-0.08 \cdot x}}$", ha="left", va="center", fontsize=16)

for ax in [ax1, ax12]:
    ax.set_ylabel("Time elapsed (minutes)", labelpad=20)
    ax.set_ylim(0, 17)
    ax.set_yticks(np.arange(0.5, 16.75, 1))
    ax.set_yticklabels(np.arange(-1, 16, 1)[::-1])
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xticks([])
    ax.set_xlim(0, 1)
    vals = ax.get_yticks()
    for i, val in enumerate(vals[::-1]):
        ax.axhline(y=val, xmin=-0.05, xmax=0, color="k", alpha=0.5, 
                   zorder=1, clip_on=False)
        if i == 0:
            valtext = "P"
        else:
            valtext = str(15-int(val-0.5))
        ax.text(-0.125, val, valtext, ha="right", va="center")
    ax.set_yticks([])
    ax.spines['left'].set_visible(False)
    
for n in range(0, len(data)):
    row = int(4+np.floor(n/10))
    col = int(2+n%10)
    ax = fig.add_subplot(grid[row, col], facecolor="None", aspect="equal")
    ax.imshow(data[n], cmap=cmap, vmin=0, vmax=100)
    ax.set_xlim(0, 80-1)
    ax.set_ylim(30-1, 0)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['bottom'].set_color(EL.c_greyax)
    ax.spines['top'].set_color(EL.c_greyax) 
    ax.spines['right'].set_color(EL.c_greyax)
    ax.spines['left'].set_color(EL.c_greyax)
    ax.set_aspect("equal")
    
for i, time in enumerate(times):
    name = './data/fluorescein/larvae_averages_1mm/'+time+'_averages.csv'
    df = pd.read_csv(name, header=None)
    row = int(4+i)
    ax = fig.add_subplot(grid[row, 0], facecolor="None", aspect="equal")
    ax.imshow(df, vmin=0, vmax=100, cmap=cmap)
    ax.set_xlim(0, 80-1)
    ax.set_ylim(30-1, 0)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['bottom'].set_color(EL.c_greyax)
    ax.spines['top'].set_color(EL.c_greyax) 
    ax.spines['right'].set_color(EL.c_greyax)
    ax.spines['left'].set_color(EL.c_greyax)
    ax.set_aspect("equal")

# SET BOUNDARIES AND SAVE FIGURE -----------------------------------------------
plt.tight_layout()
fig.subplots_adjust(wspace=0.1, hspace=0.1)
pp = pdf.PdfPages(savename, keep_empty=False)
pp.savefig(fig)
pp.close()
plt.show()